In [ ]:
import Flux

In [ ]:
# utility functions
struct Utility
    W::Any
    b::Any
    mode::Any
end

function Utility(N::Integer, mode::String; init = Flux.glorot_uniform)
    if mode in ["log cobb-douglas", "substitutes"]
        Utility(init(1, N), nothing, mode)
    elseif mode in ["log ces +"]
        Utility(init(1, N), init(1), mode)
    elseif mode in ["log ces -"]
        Utility(init(1, N), -exp.(init(1)), mode)
        @assert false
    end
end

function (m::Utility)(x::AbstractVecOrMat)
    x = max.(x, 1f-10) # for now, assume that goods take on positive values
    W = exp.(m.W)
    if m.mode == "log cobb-douglas"
        return W * log.(x)
    elseif m.mode in ["log ces +", "log ces -"]
        b = m.b
        if m.mode == "log ces +"
            b = @. Flux.sigmoid(m.b)
        end
        log.(sum((W' .* x) .^ b, dims = 1)') ./ b
    elseif m.mode == "substitutes"
        return W * x
    else
        @assert false
    end
end

(m::Utility)(x::AbstractArray) = reshape(m(reshape(x, size(x, 1), :)), :, size(x)[2:end]...)
Flux.@functor Utility

In [ ]:
Join(combine, paths) = Flux.Parallel(combine, paths)
Join(combine, paths...) = Join(combine, paths)
struct Split{T}
    paths::T
end
Split(paths...) = Split(paths)
Flux.@functor Split
(m::Split)(x::AbstractArray) = map(f -> f(x), m.paths)